In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import odym.classes as msc
import odym.functions as msf
import odym.dynamic_stock_model as dsm

In [4]:
ModelClassification  = {} # Create dictionary of model classifications

ModelClassification['Time'] = msc.Classification(
    Name='Time',
    Dimension='Time',
    ID=1,
    Items=list(np.arange(2015,2025))
)

ModelClassification['Element'] = msc.Classification(
    Name='Fibers',
    Dimension='Element',
    ID=2,
    Items=["Acrylic", "Cotton", "Polyamide/nylon", "Polyester", "Silk", "Viscose", "Wool", "Animal hair"]
)

ModelClassification["Clothing"] = msc.Classification(
    Name='Clothing Type',
    Dimension='Clothing',
    ID=3,
    Items = ['shirts 1', 'shirts 2', 'underwear', 'sportswear', 'handkerchiefs 1', 'handkerchiefs 2', 't-shirts,', 'trousers', 'sweaters', 'overcoats'],
)   

Model_Time_Start = int(min(ModelClassification['Time'].Items))
Model_Time_End   = int(max(ModelClassification['Time'].Items))
Model_Duration   = Model_Time_End - Model_Time_Start

In [5]:
IndexTable = pd.DataFrame({'Aspect'        : ['Time','Element','Clothing'], # 'Time' and 'Element' must be present!
                           'Description'   : ['Model aspect "time"', 'Model aspect "fiber type"', 'Model aspect "clothing type"'],
                           'Dimension'     : ['Time','Element','Clothing'], # 'Time' and 'Element' are also dimensions
                           'Classification': [ModelClassification[Aspect] for Aspect in ['Time','Element', 'Clothing']],
                           'IndexLetter'   : ['t','e','c']}) # Unique one letter (upper or lower case) indices to be used later for calculations.

IndexTable.set_index('Aspect', inplace = True) # Default indexing of IndexTable, other indices are produced on the fly

IndexTable

,Description,Dimension,Classification,IndexLetter
Aspect,,,,
Time,"Model aspect ""time""",Time,<odym.classes.classification.Classification ob...,t
Element,"Model aspect ""fiber type""",Element,<odym.classes.classification.Classification ob...,e
Clothing,"Model aspect ""clothing type""",Clothing,<odym.classes.classification.Classification ob...,c


In [6]:
Dyn_MFA_System = msc.MFAsystem(
    Name='Clothing Cycle in Households', 
    Geogr_Scope='Denmark', 
    Unit='kt', 
    ProcessList=[], 
    FlowDict={}, 
    StockDict={},
    ParameterDict={}, 
    Time_Start=Model_Time_Start, 
    Time_End=Model_Time_End, 
    IndexTable=IndexTable, 
    Elements=IndexTable.loc['Element'].Classification.Items
)